In [1]:
import torch
import os
from torchvision.utils import save_image
from tqdm import tqdm
import torch.nn as nn

In [2]:
# --- Set device and load model ---
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# --- Define Generator Model (use your trained architecture) ---
class Generator(nn.Module):
    def __init__(self, latent_dim=100):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias=False),  # Output: 3x64x64
            nn.Tanh()
        )

    def forward(self, x):
        return self.gen(x)

In [4]:
latent_dim = 100
generator = Generator(latent_dim=latent_dim).to(device)
generator.load_state_dict(torch.load("generator.pth", map_location=device))
generator.eval()

Generator(
  (gen): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

In [5]:
# --- Generate and save images ---
os.makedirs("generated_fakes", exist_ok=True)

batch_size = 64
num_images = 6380+6380
num_batches = (num_images + batch_size - 1) // batch_size

In [6]:
with torch.no_grad():
    for batch_idx in tqdm(range(num_batches), desc="Generating fakes"):
        noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
        fake_imgs = generator(noise)

        for i, img in enumerate(fake_imgs):
            if batch_idx * batch_size + i >= num_images:
                break
            filename = f"generated_fakes/fake_{batch_idx * batch_size + i:04d}.png"
            save_image(img, filename, normalize=True)

print("✅ 1000 fake images saved in 'generated_fakes'")

Generating fakes: 100%|██████████████████████████████████████████████████████████████| 200/200 [01:23<00:00,  2.40it/s]

✅ 1000 fake images saved in 'generated_fakes'
